# SCA through Dino LFP data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/notebooks_visualization/')
# sys.path.append('/Users/Lauren/Voytek/spectralCV')
sys.path.append('/Users/rdgao/Documents/code/research/spectralCA')
sys.path.append('/Users/rdgao/Documents/code/research/neurodsp/')
# sys.path.append('/Users/rdgao/Documents/code/research/fooof/')

In [2]:
%matplotlib inline

# imports
from sca_funcs import access_nt as asc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd

import neurodsp as ndsp
# from fooof import FOOOFGroup
from sca_funcs import sca

In [3]:
#data_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_ephys.mat"
data_path = "/Users/rdgao/Documents/data/CRCNS/fcx1/Dino_072114_mPFC/Dino_072114_mPFC_ephys.mat"
dino = io.loadmat(data_path, squeeze_me=True)

In [6]:
#info_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_wakesleep.csv"
info_path = "/Users/rdgao/Documents/data/CRCNS/fcx1/Dino_072114_mPFC/Dino_072114_mPFC_wakesleep.csv"
info = pd.read_csv(info_path,index_col='Unnamed: 0')

In [10]:
info[-3:]

,Label,Start,Inter,End
175,Wake,5,NaN,3361
176,Sleep,3361,NaN,4992
177,WakeSleep,5,3361.0,4992


In [48]:
label = 'Wake'
duration = list(zip(info[info['Label'] == label]['Start'].values, info[info['Label'] == label]['End'].values))
fs = 1250
analysis_param = {'nperseg': int(fs),
                 'noverlap': 0,
                 'spg_outlierpct': 4.,
                 'max_freq':200}

data = []
for dur in duration:
    start, end = dur
    
    # don't append like this because there will be a gap if there are multiple trials
    # of the same type. Compute sca separately
    data.append(dino['lfp'][:,start*fs:end*fs])
    
data = np.column_stack(data)

In [49]:
# initialize object with data and parameters
dino_sca = sca.SCA(analysis_param)
dino_sca.populate_ts_data(data, fs)

# compute all spectral attributes
dino_sca.compute_all_spectral()

# compute KS-test fit against null exponential hypothesis
dino_sca.compute_KS_expfit() # this takes a few minutes

# save out
dino_sca.save_spec_vars('../results/dino/'+label.lower(), save_spg=True)